# 🚀 2강: Transfer Learning 실습 — MobileNetV2 + Grad-CAM

**목표**
- 사전학습(Imagenet) 기반 **MobileNetV2**를 활용하여 작은 데이터에도 빠르게 높은 성능을 얻습니다.
- **Feature Extraction → Fine-tuning** 두 단계로 학습하고, **Grad-CAM**으로 설명가능성을 더합니다.

**소요시간 가이드 (30분)**
- 도입/데이터 준비(8분), 모델/학습(15분), 평가/시각화/Grad-CAM(7분)


## 0. 환경 설정 및 라이브러리 임포트

In [ ]:
import os, sys, time
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report

print(f"TensorFlow: {tf.__version__}")
print("GPU Available:", tf.config.list_physical_devices('GPU'))


## 1. 데이터셋 로드 & 전처리 (CIFAR-10 → 224×224로 리사이즈)

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)

from sklearn.model_selection import train_test_split
x_tr, x_va, y_tr, y_va = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

IMG_SIZE = 224
def resize_images(x):
    x = tf.image.resize(x, (IMG_SIZE, IMG_SIZE)).numpy()
    return x

x_tr = resize_images(x_tr)
x_va = resize_images(x_va)
x_te = resize_images(x_test)

print('Shapes:', x_tr.shape, x_va.shape, x_te.shape)

In [ ]:
## 🔁 데이터 증강 (실시간)
data_aug = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

## 2. 사전학습 모델 — Feature Extraction 단계

In [ ]:
## 🧠 MobileNetV2 (ImageNet 사전학습) 불러오기
base = MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights='imagenet')
base.trainable = False  # Feature Extraction 단계에서는 Freeze

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_aug(inputs)
x = preprocess_input(x)  # MobileNetV2 전처리
x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(10, activation='softmax')(x)
model = models.Model(inputs, outputs)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
es = EarlyStopping(patience=2, restore_best_weights=True, monitor='val_accuracy')
hist_fe = model.fit(x_tr, y_tr, validation_data=(x_va, y_va), epochs=5, batch_size=64, callbacks=[es], verbose=1)

In [ ]:
## 📈 학습 곡선 함수
def plot_history(history, title='Training History'):
    h = history.history
    plt.figure(figsize=(6,4))
    plt.plot(h['accuracy'], label='train_acc')
    plt.plot(h['val_accuracy'], label='val_acc')
    plt.title(title); plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend(); plt.tight_layout(); plt.show()
    
    plt.figure(figsize=(6,4))
    plt.plot(h['loss'], label='train_loss')
    plt.plot(h['val_loss'], label='val_loss')
    plt.title(title); plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend(); plt.tight_layout(); plt.show()

plot_history(hist_fe, 'Feature Extraction History')

## 3. Fine-tuning 단계 (상위 블록만 재학습)

In [ ]:
## 🔓 일부 레이어 Unfreeze (상위 50개 레이어 예시)
base.trainable = True
for layer in base.layers[:-50]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])
hist_ft = model.fit(x_tr, y_tr, validation_data=(x_va, y_va), epochs=5, batch_size=64, callbacks=[EarlyStopping(patience=2, restore_best_weights=True, monitor='val_accuracy')], verbose=1)
plot_history(hist_ft, 'Fine-tuning History')

## 4. 테스트 평가 및 혼동행렬

In [ ]:
test_loss, test_acc = model.evaluate(x_te, y_test, verbose=0)
print(f"[Transfer] Test Acc: {test_acc:.4f} | Loss: {test_loss:.4f}")

y_pred = np.argmax(model.predict(x_te, verbose=0), axis=1)
cm = confusion_matrix(y_test, y_pred, labels=range(10))
plt.figure(figsize=(6,5))
plt.imshow(cm, interpolation='nearest')
plt.title('Confusion Matrix (Transfer)')
plt.colorbar(); plt.xticks(range(10), class_names, rotation=45); plt.yticks(range(10), class_names)
plt.tight_layout(); plt.xlabel('Predicted'); plt.ylabel('True'); plt.show()

print('\n[Classification Report]\n')
print(classification_report(y_test, y_pred, target_names=class_names))

## 5. 오분류 사례 시각화

In [ ]:
mis_idx = np.where(y_pred != y_test)[0]
plt.figure(figsize=(8,8))
for i, id_ in enumerate(mis_idx[:16], 1):
    plt.subplot(4,4,i)
    plt.imshow(x_te[id_].astype('uint8'))
    plt.title(f"T:{class_names[y_test[id_]]}\nP:{class_names[y_pred[id_]]}")
    plt.axis('off')
plt.tight_layout(); plt.show()

## 6. Grad-CAM으로 모델 시각적 설명 추가

In [ ]:
## 🔥 Grad-CAM 구현 함수
def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        class_idx = tf.argmax(predictions[0])
        loss = predictions[:, class_idx]
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + 1e-9)
    return heatmap.numpy()

## 📌 MobileNetV2의 마지막 Conv 레이어 이름 예: 'Conv_1'
last_conv = 'Conv_1'

## 샘플 이미지 하나 선택
idx = np.random.randint(0, len(x_te))
img = x_te[idx].astype('uint8')
img_input = np.expand_dims(img, axis=0)
img_pre = preprocess_input(img_input.astype('float32'))

## Heatmap 생성
heatmap = make_gradcam_heatmap(img_pre, model, last_conv)
heatmap = np.uint8(255 * heatmap)

## 원본 위에 오버레이
import cv2
heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
overlay = cv2.addWeighted(img, 0.6, heatmap_color, 0.4, 0)

plt.figure(figsize=(9,3))
plt.subplot(1,3,1); plt.imshow(img); plt.title('Original'); plt.axis('off')
plt.subplot(1,3,2); plt.imshow(heatmap, cmap='jet'); plt.title('Grad-CAM Heatmap'); plt.axis('off')
plt.subplot(1,3,3); plt.imshow(overlay[..., ::-1]); plt.title('Overlay'); plt.axis('off')
plt.tight_layout(); plt.show()

## 7. 모델 저장/로딩 및 실전 팁

In [ ]:
model.save('transfer_mobilenetv2_finetuned.keras')
print('모델 저장 완료: transfer_mobilenetv2_finetuned.keras')

loaded = tf.keras.models.load_model('transfer_mobilenetv2_finetuned.keras')
print('로딩 테스트:', loaded.evaluate(x_te, y_test, verbose=0))

### ✅ 실전 팁
- 작은 데이터셋에서는 **Feature Extraction** 만으로도 충분히 높은 성능을 얻을 수 있음.
- Fine-tuning은 **상위 몇 개 블록만** 열고 학습률을 **아주 작게** 설정하세요 (예: 1e-5).
- 입력 해상도를 모델 권장값(224×224)로 맞추고, 전처리 함수(`preprocess_input`)를 꼭 사용하세요.
- 설명가능성(Grad-CAM)을 통해 **모델이 어디를 보고 판단하는지** 확인하면 품질 관리에 큰 도움이 됩니다.
